In [28]:
from fastai.tabular.all import *
from fastai.collab import *

import ipywidgets as widgets
from IPython.core.display import HTML, display
from ipywidgets import  Layout, HBox, VBox


In [29]:
jokes = pd.read_csv('./jester_items.csv')
ratings = pd.read_csv('./jester_ratings.csv') 
ratings = ratings.merge(jokes) 
#ratings.head()

In [30]:
#Load learner from 
path = Path('./recommender.pkl')
learn = load_learner(path)

In [31]:
#Global variables:
uniqueJokeIds = ratings.jokeId.unique()

randId = -99

ids,ratingsL = ([],[])

#Output widget:
out = widgets.Output()

#Start button:
start = widgets.Button(description='Start Rating',)

#Start Info Widgets
lt1 =  widgets.HTML("<h3><font color=''>Jester App</font><h3>")
expl = widgets.HTML("<p ><font color=''>Rate 10 jokes and the program understands your sense of humor</font><p><p>You can skip the jokes you don't wanna vote.<p>")

#Start VBox
head = VBox([lt1, expl, start])

#Rate button,Skip button
rate = widgets.Button(description='Rate',  button_style='success')
skip = widgets.Button(description='Skip',  button_style='info')

#Rate Slider
rateSlider = widgets.FloatSlider(description='Select Score:',min=-10,max=10)

#Rating row
ratingRow = HBox([rateSlider, rate, skip])

#Progress bar
ratingProcess = widgets.IntProgress(
    value=0,
    min=0,
    max=10,
    step=1,
    description='',
)

#Rating header
expl2 = widgets.HTML("<h3><font color=''>Random Joke</font><h3>")

#Waiter
expl3 = widgets.HTML("<h3><font color=''>Rating is done. Please wait a minute...</font><h3>")

#Recom Header
expl4 = widgets.HTML("<h3><font color=''>Top 3 jokes that you would enjoy the most:</font><h3>")
expl5 = widgets.HTML("<h3><font color=''>Bottom 3 jokes that you would enjoy the least</font><h3>")

rating = VBox([ratingRow,ratingProcess])


#Again
startAgain = widgets.Button(description='Start Again',button_style='danger')

In [32]:
#Start rating'e tıklanınca output temizlensin, yeni arayüz yüklensin:
def rateFunc(a):
    ratingsL.append(rateSlider.value)
    ids.append(randId)
    rateSlider.value = 0
    #global ratingProcess
    ratingProcess.value += 1
    global uniqueJokeIds
    uniqueJokeIds = np.delete(uniqueJokeIds, np.where(uniqueJokeIds == randId))
    #Eğer 10 tane joke ratelenirse, önerileri göster:
    if len(ids) <= 9:
        printRandomJoke()
    else:
        recFunc()
        
    
def skipFunc(a):
    rateSlider.value = 0
    global uniqueJokeIds
    uniqueJokeIds = np.delete(uniqueJokeIds, np.where(uniqueJokeIds == randId))
    printRandomJoke()

def printRandomJoke():
    out.clear_output()
    with out:
        display(expl2)
        global randId
        randId = random.choice(uniqueJokeIds)
        print(jokes.iloc[randId-1,1])
        display(rating)

def startFunc(a):
    out.clear_output()
    printRandomJoke()

def recFunc():
    out.clear_output()
    with out:
        display(expl3)
    recBackend()
    out.clear_output()
    with out:
        display(expl4)
        for i in range(len(topThree)):
            print(topThree[topThree["jokeId"]==jokeIndexes[i]]["jokeText"].values[0])
            print("\nPredicted rating: "+str(maxPredRatings[i]))
            print("--------------------------------------------")
        display(expl5)
        for i in range(len(botThree)):
            print(botThree[botThree["jokeId"]==minjokeIndexes[i]]["jokeText"].values[0])
            print("\nPredicted rating: "+str(minPredRatings[i]))
            print("--------------------------------------------")
        display(startAgain)

    
def recBackend():
    global ratingsL
    ratingsL = [int(i) for i in ratingsL]
    global inputDF
    inputDF = pd.DataFrame({'jokeId':ids,'InputRating':ratingsL})
    
    #Find the most similar user:
    df = ratings.drop("jokeText",axis=1);
    df = df[df['jokeId'].isin(ids)]
    df = df.sort_values(by = 'userId') 

    for i in range(len(inputDF)):
        df = df.groupby(['userId']).filter(lambda x: any(x['jokeId'] == ids[i]))
    df

    global merged
    merged = df.merge(inputDF)
    merged = merged.sort_values(by = 'userId')

    merged["err"] = abs(merged["rating"]-merged["InputRating"])

    df2 = merged.groupby(['userId'])['err'].agg('sum')

    #Here is the user id:
    mostSimilarUserId = df2.idxmin()
    
    
    #Find the Errors:
    minErr = df2[df2.idxmin()]
    minAvgErr = df2[df2.idxmin()]/len(inputDF)
    minErr,minAvgErr
    
    #See the selected user's ratings for selected jokes:

    merged = merged[merged["userId"]==mostSimilarUserId]
    
    #Offer new jokes:
    selectedUserDF = ratings[ratings["userId"]==mostSimilarUserId]
    #selectedUserDF = selectedUserDF[~selectedUserDF.jokeId.isin(ids)]
    
    
    #Top 3 jokes of selected user based on embeddings:
    Userids = [mostSimilarUserId] * len(uniqueJokeIds)
    
    data = {'userId': Userids, 'jokeId': uniqueJokeIds}

    newdf = pd.DataFrame(data)
    newdf = newdf[~newdf.jokeId.isin(ids)] #Delete the jokes, already voted for.
    #predictions
    dl = learn.dls.test_dl(newdf)
    preds = learn.get_preds(dl=dl)

    arr = preds[0].numpy()
    #Get max 3 indexes
    indexes = arr.argsort()[-3:][::-1]
    #Get max 3 ratings
    global maxPredRatings
    maxPredRatings = arr[indexes]#3 rating predictions for jokeIndexes
    #print(maxPredRatings)
    newdf2 = newdf.iloc[indexes]
    global jokeIndexes
    jokeIndexes = newdf2.iloc[:,1].values #top 3 predicted jokes
    #print(jokeIndexes)
    global topThree
    indexedJokes = ratings[ratings['jokeId'].isin(jokeIndexes)]
    topThree = indexedJokes.drop_duplicates(['jokeId'])  
    
    #bottom three jokes
    #Get min 3 indexes
    indexes = arr.argsort()[:3][::1]

    #Get max 3 ratings
    global minPredRatings
    minPredRatings = arr[indexes]

    newdf2 = newdf.iloc[indexes]
    global minjokeIndexes
    minjokeIndexes = newdf2.iloc[:,1].values
    indexedJokes2 = ratings[ratings['jokeId'].isin(minjokeIndexes)]
    global botThree
    botThree = indexedJokes2.drop_duplicates(['jokeId'])    

def startAgainFunc(a):
    ratingProcess.value = 0
    out.clear_output()
    with out:
        display(head)

In [33]:
#Events:

#Start button:
start.on_click(startFunc)

#Rate button,Skip button
rate.on_click(rateFunc)
skip.on_click(skipFunc)

#Start again button:
startAgain.on_click(startAgainFunc)

In [34]:
#Display initial output:
out.clear_output()
with out:
    display(head)
out

Output()